In [ ]:
import xarray as xr
import fsspec
import rioxarray
import numpy as np
import pandas as pd
from cmip6_downscaling.methods.common.utils import calc_auspicious_chunks_dict

In [ ]:
input_url_pattern = "s3://nex-gddp-cmip6-cog/monthly/CMIP6_ensemble_median/tasmax/tasmax_month_ensemble-median_historical_{yyyymm}.tif"

In [ ]:
dates = pd.date_range("1950-01-01", "2014-12-31", freq="M", inclusive="both")
dates
input_urls = [input_url_pattern.format(yyyymm=day.strftime("%Y%m")) for day in dates]

In [ ]:
def fix_dims(xda):
    # preprocess function fed into open_mfdataset. Extracts datetime from tiff encoding and creates time dimension.

    time = xda.encoding["source"].split("_")[-1].split(".")[0]
    xda = (
        xda.expand_dims(time=[np.datetime64(time[0:4] + '-' + time[4:6])])
        .squeeze(dim=["band"], drop=True)
        .drop("spatial_ref")
        .rename({"band_data": "tasmax", "x": "lon", "y": "lat"})
        .sortby("lat")
    )
    return xda

In [ ]:
ds = xr.open_mfdataset(
    input_urls,
    engine="rasterio",
    chunks={},
    parallel=True,
    concat_dim="time",
    combine="nested",
    preprocess=fix_dims,
)

In [ ]:
chunks_dict = calc_auspicious_chunks_dict(ds['tasmax'], chunk_dims=('time',))
chunks_dict

In [ ]:
ds = ds.chunk(chunks=chunks_dict)
ds = ds.assign_coords({"time": ds.time.astype("datetime64[ns]")})
ds

In [ ]:
import os

target_root = (
    "s3://carbonplan-benchmarks/data/c0/nex-gddp-cmip6/monthly/CMIP6_ensemble_median/tasmax/"
)
store_name = "tasmax_month_ensemble-median_historical.zarr"
target_store = os.path.join(target_root, store_name)
target_store

In [ ]:
ds.to_zarr(target_store, consolidated=True, mode="w")